In [2]:
#import the necessary libraries
import pandas as pd   
from matplotlib import pyplot
import numpy as np
import time


from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

In [6]:
df = pd.read_csv('train.csv')#reading the csv file train.csv

In [7]:
df_onehot = pd.get_dummies(df)
df_onehot.dtypes

RecordID                                     int64
hospital_id                                  int64
icu_id                                       int64
age                                        float64
elective_surgery                             int64
                                            ...   
apache_2_bodysystem_Renal/Genitourinary      uint8
apache_2_bodysystem_Respiratory              uint8
apache_2_bodysystem_Trauma                   uint8
apache_2_bodysystem_Undefined Diagnoses      uint8
apache_2_bodysystem_Undefined diagnoses      uint8
Length: 96, dtype: object

In [8]:
df_onehot.fillna(df_onehot.mean(), inplace=True)

In [9]:
X = df_onehot.loc[:, df_onehot.columns != 'hospital_death']
y = df_onehot[['hospital_death']]

In [227]:
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=2)

In [228]:
from sklearn.feature_selection import SelectKBest, f_classif
best_sel = SelectKBest(score_func=f_classif, k=10).fit(X,y)
trainX = best_sel.transform(trainX)
testX = best_sel.transform(testX)

c:\Users\Zuhair Shah\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [236]:
col = best_sel.get_support(indices=True)


In [238]:
df_new = df.iloc[:,col]

In [239]:
df_new.columns

Index(['apache_2_bodysystem', 'age', 'pre_icu_los_days', 'gcs_motor_apache',
       'ventilated_apache', 'd1_diasbp_min', 'd1_diasbp_noninvasive_min',
       'd1_heartrate_max', 'h1_spo2_min', 'h1_sysbp_max'],
      dtype='object')

In [229]:
X = best_sel.transform(X)

In [10]:
%%timeit -n1 -r1
parameters = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}
et = ExtraTreesClassifier()
cv = RepeatedKFold(n_splits=10, n_repeats=1)#, random_state=1)ler
grid_search = GridSearchCV(et, parameters, cv=cv, n_jobs=-1, scoring='roc_auc',verbose=2)#, refit=False)
grid_search.fit(X, y)
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 54 candidates, totalling 540 fits


c:\Users\Zuhair Shah\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


ExtraTreesClassifier(max_features=10, min_samples_leaf=3, min_samples_split=10,
                     n_estimators=300)
0.8725089728473225
{'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 10, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
1h 17min 50s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [231]:
dt_model = DecisionTreeClassifier(max_depth=7, min_samples_split=10)

In [232]:
dt_model

DecisionTreeClassifier(max_depth=7, min_samples_split=10)

In [233]:
df1 = pd.read_csv('test.csv')

In [234]:
df1_onehot = pd.get_dummies(df1)
df1_onehot.dtypes

RecordID                                     int64
hospital_id                                  int64
icu_id                                       int64
age                                        float64
elective_surgery                             int64
                                            ...   
apache_2_bodysystem_Renal/Genitourinary      uint8
apache_2_bodysystem_Respiratory              uint8
apache_2_bodysystem_Trauma                   uint8
apache_2_bodysystem_Undefined Diagnoses      uint8
apache_2_bodysystem_Undefined diagnoses      uint8
Length: 95, dtype: object

In [240]:
df1_onehot = df1[['apache_2_bodysystem', 'age', 'pre_icu_los_days', 'gcs_motor_apache',
       'ventilated_apache', 'd1_diasbp_min', 'd1_diasbp_noninvasive_min',
       'd1_heartrate_max', 'h1_spo2_min', 'h1_sysbp_max']]

In [243]:
x1 = df1_onehot.fillna(df1_onehot.mean(), inplace=True)

C:\Users\Zuhair Shah\AppData\Local\Temp\ipykernel_28500\3342560244.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  x1 = df1_onehot.fillna(df1_onehot.mean(), inplace=True)
C:\Users\Zuhair Shah\AppData\Local\Temp\ipykernel_28500\3342560244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1 = df1_onehot.fillna(df1_onehot.mean(), inplace=True)
